In [44]:
#standard ds imports
import pandas as pd
import numpy as np
#viz and stats
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
# .py imports
import wrangle_g as wr
#import modeling as md
import os
import nltk
import unicodedata
import re
#sklearn imports
from sklearn.model_selection import train_test_split
import sklearn.preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, accuracy_score,confusion_matrix, plot_confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
#CATboost imports
from catboost import CatBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

In [2]:
df = pd.read_csv('lang_data_prepped.csv')

In [3]:
df

,repo,language,readme_contents,stopped
0,ujjwalkarn/DataScienceR,R,# R Data Science Tutorials\n- This repo contai...,data science tutorials repo contain curated li...
1,IndrajeetPatil/ggstatsplot,R,---\noutput: github_document\n---\n\n <!-- RE...,output githubdocument readmemd generate readme...
2,easystats/easystats,R,"---\noutput: github_document\n---\n\n```{r, wa...",output githubdocument warningfalse messagefals...
3,seandavi/sars2pack,R,---\noutput: \n - rmarkdown::github_document\...,output rmarkdowngithubdocument rmarkdownhtmldo...
4,markvanderloo/lumberjack,R,## Track changes in data\n[![Build Status](htt...,track change data build statushttpstravisciorg...
...,...,...,...,...
286,rahulbordoloi/Machine-Learning,Python,# Machine-Learning\nContains my Glossary for M...,machinelearning contain glossary machine learn...
287,Oslandia/QDeeplandia,Python,QGIS plugin dedicated to 2D semantic segmentat...,qgis plugin dedicate 2d semantic segmentation ...
288,arjunbhasin2013/BaselinePredictionMLFramework,Python,# Baseline Prediction Machine Learning Framewo...,baseline prediction machine learn framework ma...
289,PonderaLab/datascience4economists,Python,"# Datascience4economists\n\nFirst of all, welc...",datascience4economists first welcome repositor...


In [4]:
df = pd.get_dummies(df, columns=['language'], drop_first=True)

In [5]:
df.head()

,repo,readme_contents,stopped,language_R
0,ujjwalkarn/DataScienceR,# R Data Science Tutorials\n- This repo contai...,data science tutorials repo contain curated li...,1
1,IndrajeetPatil/ggstatsplot,---\noutput: github_document\n---\n\n <!-- RE...,output githubdocument readmemd generate readme...,1
2,easystats/easystats,"---\noutput: github_document\n---\n\n```{r, wa...",output githubdocument warningfalse messagefals...,1
3,seandavi/sars2pack,---\noutput: \n - rmarkdown::github_document\...,output rmarkdowngithubdocument rmarkdownhtmldo...,1
4,markvanderloo/lumberjack,## Track changes in data\n[![Build Status](htt...,track change data build statushttpstravisciorg...,1


## BASELINE-using all datapoints

In [6]:
df.language_R.value_counts()

0    167
1    124
Name: language_R, dtype: int64

In [7]:
df['baseline'] = 0

In [8]:
baseline_accuracy = (df.baseline == df.language_R).mean()
print(f'baseline accuracy: {baseline_accuracy:.2%}')

baseline accuracy: 57.39%


In [9]:
subset = df[df.language_R == 0]
baseline_recall = (subset.baseline == subset.language_R).mean()
print(f'baseline recall: {baseline_recall:.2%}')

baseline recall: 100.00%


In [10]:
subset = df[df.baseline == 0]
baseline_precision = (subset.baseline == subset.language_R).mean()
print(f'baseline precision: {baseline_precision:.2%}')

baseline precision: 57.39%


In [11]:
df.drop(columns='baseline',inplace=True)

## Train/Validate/Test Split

In [12]:
df.rename(columns = {'language_R':'language'}, inplace = True)

In [13]:
train, validate, test = wr.final_wrangle(df)

In [14]:
train.shape[0],validate.shape[0],test.shape[0]

(162, 70, 59)

## Create a column showing token count

In [15]:
train['token_cnt'] = [len(row['stopped'].split()) for _, row in train.iterrows()]

In [16]:
train.head()

,repo,readme_contents,stopped,language,token_cnt
117,storieswithsiva/US-Job-Market-Analysis,# US Job Analysis in R\n[![Makes people smile]...,us job analysis make people smilehttpsfortheba...,1,381
8,adityashrm21/Bike-Sharing-Demand-Kaggle,# Bike-Sharing-Demand---Kaggle\n\nSolution to ...,bikesharingdemandkaggle solution kaggle knowle...,1,12
63,Sarah-Hesham-2022/Bio-Statistics-R-Coding-On-D...,# Bio-Statistics-R-Coding-On-Datasets-Analysis...,biostatisticsrcodingondatasetsanalysisandplott...,1,35
268,Tech-Matt/game_of_life,# Game of Life\n\n## Premise\nThis is the firs...,game life premise first readme im write projec...,0,125
105,vincenzorusso3/tesi-triennale,# PPMI Research\nThe main goal is to create a ...,ppmi research main goal create system order cl...,1,78


## rename column to language_R

In [17]:
train.rename(columns = {'language':'language_R'}, inplace = True)
validate.rename(columns = {'language':'language_R'}, inplace = True)
test.rename(columns = {'language':'language_R'}, inplace = True)

## remove extraneous clutter cols

In [18]:
train = train.drop(columns=['repo','readme_contents'])
validate = validate.drop(columns=['repo','readme_contents'])
test = test.drop(columns=['repo','readme_contents'])

In [19]:
train.head()

,stopped,language_R,token_cnt
117,us job analysis make people smilehttpsfortheba...,1,381
8,bikesharingdemandkaggle solution kaggle knowle...,1,12
63,biostatisticsrcodingondatasetsanalysisandplott...,1,35
268,game life premise first readme im write projec...,0,125
105,ppmi research main goal create system order cl...,1,78


In [20]:
train.language_R.value_counts()

0    93
1    69
Name: language_R, dtype: int64

## create X and y train/validate/test

In [33]:
    # split train into X (dataframe, drop target) & y (series, keep target only)
    X_train = train['stopped']
    y_train = train['language_R']
    
    # split validate into X (dataframe, drop target) & y (series, keep target only)
    X_validate = validate['stopped']
    y_validate = validate['language_R']
    
    # split test into X (dataframe, drop target) & y (series, keep target only)
    X_test = test['stopped']
    y_test = test['language_R']

## TF-IDF

In [34]:
tfidf = TfidfVectorizer()

In [35]:
X_bow = tfidf.fit_transform(X_train)

## MODELING NOW

### DTC maxDepth = 4

In [36]:
# create the Decision Tree Classifier model 
dtc = DecisionTreeClassifier(max_depth=4)

In [37]:
# fit the model to the TRAIN dataset:
dtc.fit(X_bow, y_train)
dtc.score(X_bow,y_train)

0.8209876543209876

In [38]:
dtc_preds = dtc.predict(X_bow)
pd.crosstab(dtc_preds,y_train)

language_R,0,1
row_0,,
0,64,0
1,29,69


In [40]:
# as with any other sklearn transformation, 
# transform only on our validate and/or test, 
# only fit on train
X_validate_bow = tfidf.transform(X_validate)
dtc.score(X_validate_bow, y_validate)

0.7714285714285715

In [42]:
print(f'Accuracy-Train {round(dtc.score(X_bow,y_train),4)}')
print(f'Accuracy-Validate {round(dtc.score(X_validate_bow,y_validate),4)}')
print(classification_report(y_train,dtc_preds))
print(classification_report(y_validate,dtc.predict(X_validate_bow)))

Accuracy-Train 0.821
Accuracy-Validate 0.7714
              precision    recall  f1-score   support

           0       1.00      0.69      0.82        93
           1       0.70      1.00      0.83        69

    accuracy                           0.82       162
   macro avg       0.85      0.84      0.82       162
weighted avg       0.87      0.82      0.82       162

              precision    recall  f1-score   support

           0       0.90      0.68      0.77        40
           1       0.68      0.90      0.77        30

    accuracy                           0.77        70
   macro avg       0.79      0.79      0.77        70
weighted avg       0.80      0.77      0.77        70



### Multinomial Naive Bayes

In [63]:
mnb = MultinomialNB(alpha = 0.1)
mnb.fit(X_bow, y_train)
mnb.score(X_bow, y_train)

1.0

In [64]:
mnb_preds = mnb.predict(X_bow)
pd.crosstab(mnb_preds,y_train)

language_R,0,1
row_0,,
0,93,0
1,0,69


In [65]:
X_validate_bow = tfidf.transform(X_validate)
mnb.score(X_validate_bow, y_validate)

0.7428571428571429

### RFC maxDepth=6

In [78]:
rf6 = RandomForestClassifier(n_estimators=201,max_depth=2,min_samples_leaf=2)
rf6.fit(X_bow, y_train)
rf6.score(X_bow, y_train)

0.6481481481481481

In [79]:
rf6_preds = rf6.predict(X_bow)
pd.crosstab(rf6_preds,y_train)

language_R,0,1
row_0,,
0,93,57
1,0,12


In [80]:
X_validate_bow = tfidf.transform(X_validate)
rf6.score(X_validate_bow, y_validate)

0.6

In [81]:
print(f'Accuracy-Train {round(rf6.score(X_bow,y_train),4)}')
print(f'Accuracy-Validate {round(rf6.score(X_validate_bow,y_validate),4)}')
print(classification_report(y_train,rf6_preds))
print(classification_report(y_validate,rf6.predict(X_validate_bow)))

Accuracy-Train 0.6481
Accuracy-Validate 0.6
              precision    recall  f1-score   support

           0       0.62      1.00      0.77        93
           1       1.00      0.17      0.30        69

    accuracy                           0.65       162
   macro avg       0.81      0.59      0.53       162
weighted avg       0.78      0.65      0.57       162

              precision    recall  f1-score   support

           0       0.59      1.00      0.74        40
           1       1.00      0.07      0.12        30

    accuracy                           0.60        70
   macro avg       0.79      0.53      0.43        70
weighted avg       0.76      0.60      0.48        70



### DTC maxDepth = 4 TEST

In [84]:
# as with any other sklearn transformation, 
# transform only on our validate and/or test, 
# only fit on train
X_test_bow = tfidf.transform(X_test)
dtc.score(X_test_bow, y_test)

0.6779661016949152

In [86]:
print(f'Accuracy-Train {round(dtc.score(X_bow,y_train),4)}')
print(f'Accuracy-test {round(dtc.score(X_test_bow,y_test),4)}')
print(classification_report(y_train,dtc_preds))
print(classification_report(y_test,dtc.predict(X_test_bow)))

Accuracy-Train 0.821
Accuracy-test 0.678
              precision    recall  f1-score   support

           0       1.00      0.69      0.82        93
           1       0.70      1.00      0.83        69

    accuracy                           0.82       162
   macro avg       0.85      0.84      0.82       162
weighted avg       0.87      0.82      0.82       162

              precision    recall  f1-score   support

           0       0.89      0.50      0.64        34
           1       0.57      0.92      0.71        25

    accuracy                           0.68        59
   macro avg       0.73      0.71      0.67        59
weighted avg       0.76      0.68      0.67        59



### BIGRAMS with DTC mD=4

In [87]:
tfidf2 = TfidfVectorizer(ngram_range=(2,2))
X_bow = tfidf2.fit_transform(X_train)

In [88]:
dtc = DecisionTreeClassifier(max_depth=4)
dtc.fit(X_bow, y_train)
dtc.score(X_bow, y_train)

0.7469135802469136

In [90]:
dtc_preds = dtc.predict(X_bow)
pd.crosstab(dtc_preds,y_train)

language_R,0,1
row_0,,
0,52,0
1,41,69


In [91]:
# as with any other sklearn transformation, 
# transform only on our validate and/or test, 
# only fit on train
X_validate_bow = tfidf2.transform(X_validate)
dtc.score(X_validate_bow, y_validate)


0.7714285714285715

In [92]:
print(f'Accuracy-Train {round(dtc.score(X_bow,y_train),4)}')
print(f'Accuracy-Validate {round(dtc.score(X_validate_bow,y_validate),4)}')
print(classification_report(y_train,dtc_preds))
print(classification_report(y_validate,dtc.predict(X_validate_bow)))

Accuracy-Train 0.7469
Accuracy-Validate 0.7714
              precision    recall  f1-score   support

           0       1.00      0.56      0.72        93
           1       0.63      1.00      0.77        69

    accuracy                           0.75       162
   macro avg       0.81      0.78      0.74       162
weighted avg       0.84      0.75      0.74       162

              precision    recall  f1-score   support

           0       0.96      0.62      0.76        40
           1       0.66      0.97      0.78        30

    accuracy                           0.77        70
   macro avg       0.81      0.80      0.77        70
weighted avg       0.83      0.77      0.77        70



In [93]:
# as with any other sklearn transformation, 
# transform only on our validate and/or test, 
# only fit on train
X_test_bow = tfidf2.transform(X_test)
dtc.score(X_test_bow, y_test)

0.6271186440677966

In [94]:
print(f'Accuracy-Train {round(dtc.score(X_bow,y_train),4)}')
print(f'Accuracy-test {round(dtc.score(X_test_bow,y_test),4)}')
print(classification_report(y_train,dtc_preds))
print(classification_report(y_test,dtc.predict(X_test_bow)))

Accuracy-Train 0.7469
Accuracy-test 0.6271
              precision    recall  f1-score   support

           0       1.00      0.56      0.72        93
           1       0.63      1.00      0.77        69

    accuracy                           0.75       162
   macro avg       0.81      0.78      0.74       162
weighted avg       0.84      0.75      0.74       162

              precision    recall  f1-score   support

           0       0.93      0.38      0.54        34
           1       0.53      0.96      0.69        25

    accuracy                           0.63        59
   macro avg       0.73      0.67      0.61        59
weighted avg       0.76      0.63      0.60        59

